## Understanding reservoir water levels using OPERA DSWx-HLS data

The Bhakra Nangal dam and Gobind Sagar reservoir in India [[1]](https://en.wikipedia.org/wiki/Bhakra_Dam) was opened in 1963 and provides irrigation to 10 million acres in neighboring states of Punjab, Haryana and Rajasthan. We can use the DSWx data to observe fluctutations in water levels over long time periods.

In [ ]:
import rasterio
import rioxarray
from rasterio.crs import CRS
from rasterio.warp import transform_bounds
import geoviews as gv
from geoviews import opts

import hvplot.xarray  # noqa

from shapely.geometry import Point
from osgeo import gdal

import pandas as pd

# STAC imports to retrieve cloud data
from pystac_client import Client

from datetime import datetime
import numpy as np

from util_functions import search_to_df, urls_to_dataset

from warnings import filterwarnings
filterwarnings("ignore") # suppress PySTAC warnings

# GDAL setup for accessing cloud data
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF, TIFF')

In [ ]:
bhakra_dam = (76.46, 31.42)

ref_crs = CRS.from_epsg(4326)
dst_crs = CRS.from_epsg(3857)
map_bounds = transform_bounds(ref_crs, dst_crs, *Point(*bhakra_dam).buffer(2).bounds)

In [ ]:
bhakra_dam_gv = gv.Points([bhakra_dam])

basemap = gv.tile_sources.OSM
plot = (bhakra_dam_gv*basemap).opts(
    opts.Points(
        color='red',
        alpha=0.75,
        size=25,
        width=800,
        height=800,
        xlim=(map_bounds[0], map_bounds[2]),
        ylim=(map_bounds[1], map_bounds[3]))
)
plot

In [ ]:
# We will query the DSWx product record to understand variations in water levels in the reservoir
start_date = datetime(year=2023, month=4, day=1)
stop_date = datetime(year=2024, month=4, day=1)
date_range = f'{start_date.strftime("%Y-%m-%d")}/{stop_date.strftime("%Y-%m-%d")}'

# We open a client instance to search for data, and retrieve relevant data records
STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'

# Setup PySTAC client
# POCLOUD refers to the PO DAAC cloud environment that hosts earth observation data
catalog = Client.open(f'{STAC_URL}/POCLOUD/') 

# Setup PySTAC client
provider_cat = Client.open(STAC_URL)
catalog = Client.open(f'{STAC_URL}/POCLOUD/')
collections = ["OPERA_L3_DSWX-HLS_V1"]

# Setup search options
opts = {
    'bbox' : Point(*bhakra_dam).buffer(0.01).bounds, 
    'collections': collections,
    'datetime' : date_range,
}

# Execute the search
search = catalog.search(**opts)
results = list(search.items_as_dicts())
print(f"Number of tiles found intersecting given AOI: {len(results)}")

In [ ]:
def filter_search_by_cc(results, cloud_threshold=10):
    '''
    Given a list of results returned by the NASA Earthdata STAC API for OPERA DSWx data,
    filter them by cloud cover

    The DSWx data does not always have cloud cover in the metadata. When this is the case,
    read the image and calculate the cloud fraction and apply the threshold
    '''

    filtered_results = []

    for result in results:
        try:
            cloud_cover = result['properties']['eo:cloud_cover']
        except:
            href = result['assets']['0_B01_WTR']['href']
            with rasterio.open(href) as ds:
                img = ds.read(1).flatten()
                cloud_cover = 100*(np.sum(np.isin(img, 253))/img.size)

        if  cloud_cover <= cloud_threshold:
            filtered_results.append(result)

    return filtered_results

In [ ]:
# let's filter our results so that only scenes with less than 10% cloud cover are returned
results = filter_search_by_cc(results)

print("Number of results containing less than 10% cloud cover: ", len(results))

In [ ]:
# Load results into dataframe
granules = search_to_df(results, layer_name='0_B01_WTR')

In [ ]:
dataset= urls_to_dataset(granules)

In [ ]:
# Define a colormap
COLORS = [(150, 150, 150, 0.1)]*256 # setting all colors to gray with low opacity
COLORS[0] = (0, 255, 0, 0.1) # Setting not water class to green
COLORS[1] = (0, 0, 255, 1) # Open surface water
COLORS[2] = (0, 0, 255, 1) # Partial surface water

In [ ]:
img = dataset.hvplot.image(title = 'Bhakra Nangal Dam, India - water extent over a year',
                            x='lon', y='lat', 
                            project=True, rasterize=True, 
                            tiles = gv.tile_sources.OSM,
                            cmap=COLORS, 
                            colorbar=False,
                            widget_location='bottom',
                            xlabel='Longitude (degrees)',ylabel='Latitude (degrees)',
                            fontscale=1.25, frame_width=1000, frame_height=1000)


img